In [88]:
import pandas as pd
import matplotlib as plt
import numpy as np
import time
from bs4 import BeautifulSoup

In [89]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [90]:
bestsellers = pd.read_csv(r"C:\Users\melan\Documents\DA8\Projects\bestseller_capstone\data\all_bestsellers.csv")

C:\Users\melan\AppData\Local\Temp\ipykernel_26056\3798271838.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  bestsellers = pd.read_csv(r"C:\Users\melan\Documents\DA8\Projects\bestseller_capstone\data\all_bestsellers.csv")


In [91]:
bestsellers.head()

,title,description,author,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,week,...,title_y,author_y,avg_rating,count_rating,count_reviews,date_published,top_genre,genre1,genre2,genre3
0,Full Package,A man shares a cramped apartment with his frie...,Lauren Blakely,Lauren Blakely,A00B01MT5HMRV,None,1,0,1,2017-01-25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Guernsey Literary And Potato Peel Pie Society,"After World War II, a journalist travels to th...",Mary Ann Shaffer and Annie Barrows,Dial,9780440337973,0440337976,2,0,2,2017-01-25,...,The Guernsey Literary and Potato Peel Pie Society,Mary Ann Shaffer,4.19,721465.0,60356.0,2008-07-29,"Historical Fiction, Fiction, Romance, Historic...",Historical Fiction,Fiction,Romance
2,The Whistler,A whistleblower alerts a Florida investigator ...,John Grisham,Doubleday,9780385541206,0385541201,3,4,12,2017-01-25,...,The Whistler,John Grisham,3.95,120472.0,7710.0,2016-10-25,"Fiction, Mystery, Thriller, Audiobook, Crime, ...",Fiction,Mystery,Thriller
3,Ring Of Fire,"Pike Logan, a member of a secret counterterror...",Brad Taylor,Dutton,9781101984772,None,4,0,1,2017-01-25,...,Ring of Fire,Brad Taylor,4.28,4371.0,230.0,2017-01-10,"Thriller, Fiction, Action, Military Fiction, S...",Thriller,Fiction,Action
4,Small Great Things,"A medical crisis entangles a black nurse, a wh...",Jodi Picoult,Ballantine,9780345544964,034554496X,5,7,9,2017-01-25,...,Small Great Things,Jodi Picoult,4.35,342790.0,30250.0,2016-10-11,"Fiction, Contemporary, Audiobook, Adult, Adult...",Fiction,Contemporary,Audiobook


In [92]:
#identify rows with missing goodreads data
goodreads_nulls = bestsellers[bestsellers['title_y'].isnull()]
len(goodreads_nulls)

6843

In [93]:
#set up chrome, sign in to goodreads
s = Service("chromedriver.exe")
opts= Options()
opts.headless = True
driver = webdriver.Chrome(service=s)
driver.get("https://www.goodreads.com")
#find sign in link
link = driver.find_element(By.LINK_TEXT, 'Sign In')
link.click()
#sign in with email
link2 = driver.find_element(By.LINK_TEXT, 'Sign in with email')
link2.click()
#enter username and pwd
log_email = driver.find_element(By.NAME, 'email')
log_pwd = driver.find_element(By.NAME, 'password')
log_email.send_keys("melaniedwesson@gmail.com")
log_pwd.send_keys("qK5YGNb4x4_Vp3G")
log_pwd.submit
#click sign in button
button1 = driver.find_element(By.ID, 'signInSubmit')
button1.click()

C:\Users\melan\AppData\Local\Temp\ipykernel_26056\1531218307.py:4: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  opts.headless = True


In [161]:
# #try a search by title - tried doing a title/author combo earlier and was getting too many errors
# #get unique titles, turn into a list
# unique_titles = goodreads_nulls['title'].unique()
# book_list = unique_titles.tolist()
# book_list


['Full Package',
 'Shelter For Adeline',
 'Until Ashlyn',
 'Melt',
 'Craving',
 'A Game Of Thrones',
 'Over The Edge',
 'Managed',
 'Rescuing Harley',
 'Wrong',
 'The Sexy One',
 'The Fix Up',
 'Possession',
 'Playboy Pilot',
 'A Summer In Sonoma',
 'Well Hung',
 'Anti-Stepbrother',
 'The Memory Box',
 'A Beautiful Funeral',
 'Shadow Born',
 'Midnight Soul',
 'Every Beat Of My Heart',
 'Smooth Operator',
 'Magic And Mayhem',
 'Scorch',
 'Obsession',
 'Rescuing Emily',
 'Unexpected Reality',
 'Hitched',
 'Easy Kisses',
 'Until June',
 'Six John Jordan Mysteries',
 'Thirty-Six And A Half Motives',
 'Mister O',
 'The Winslow Brothers Boxed Set',
 'Stuck-Up Suit',
 'Logan Kade',
 'The Arrangement 21 (The Ferro Family)',
 'Roomhate',
 'The Lie',
 'January',
 'April',
 'The Baller',
 'February',
 'Sebring',
 'Big Rock',
 'March',
 'Seducing Simon',
 'Monster ________',
 'Easy Melody',
 'Thirty-Five And A Half Consipiracies',
 'Hard Beat',
 'Kian',
 'Wicked After Dark',
 'Until Jax',
 'Screwe

In [95]:
# # Set up scraper loop
# self_published = []
# unavailable = []

# # Loop through book_list
# for title in book_list:
#     try:
#         # Look up each book in the search bar
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title)  # Use 'title' variable instead of undefined 'item'
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         #Click cover art
#         cover_element = driver.find_element(By.CLASS_NAME, "bookCover")
#         cover_element.click()

#         #Wait for results to load
#         time.sleep(3)

#         # Scrape the desired information
#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#         gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#         count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#         count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#         date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#         # Scrape the genre
#         genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#         genres = [genre.text.strip() for genre in genre_elements]
#         genre1 = ', '.join(genres) if genres else None

#         # Append scraped data to the self_published list
#         self_published.append({
#             'title': gr_title,
#             'author': gr_author,
#             'avg_rating': avg_rating,
#             'count_rating': count_rating,
#             'count_reviews': count_reviews,
#             'date_published': date_published,
#             'top_genre': genre1
#         })

#     except Exception as e:
#         # Handle the exception here
#         unavailable.append({
#             'title': title,  # Use 'title' variable instead of undefined 'gr_title'
#             'author': None  # Set author as None if not found
#         })


In [96]:
# len(self_published)

In [97]:
# len(unavailable)

In [98]:
# #Got some weird results in there, maybe I should try having the scraper find the link with the matching title
# # Set up scraper loop
# self_published = []
# unavailable = []

# # Loop through book_list
# for title in book_list[:10]:
#     try:
#         # Look up each book in the search bar
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title)
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find the link with a matching title
#         link_element = driver.find_element(By.XPATH, f"//a[contains(text(), '{title}')]")
#         link_element.click()

#         # Wait for the book page to load
#         time.sleep(3)

#         # Scrape the desired information
#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#         gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#         count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#         count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#         date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#         # Scrape the genre
#         genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#         genres = [genre.text.strip() for genre in genre_elements]
#         genre1 = ', '.join(genres) if genres else None

#         # Append scraped data to the self_published list
#         self_published.append({
#             'title': gr_title,
#             'author': gr_author,
#             'avg_rating': avg_rating,
#             'count_rating': count_rating,
#             'count_reviews': count_reviews,
#             'date_published': date_published,
#             'top_genre': genre1
#         })

#     except Exception as e:
#         # Handle the exception here
#         unavailable.append({
#             'title': title,
#             'author': None  # Set author as None if not found
#         })


In [99]:
# #Try again with XPATH + span
# # Set up scraper loop
# self_published = []
# unavailable = []

# # Loop through book_list
# for title in book_list[:10]:
#     try:
#         # Look up the book page URL for the given title
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title)  # Use 'title' variable instead of undefined 'item'
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find the link with a matching title
#         link_element = driver.find_element(By.XPATH, f"//span[@itemprop='name' and contains(text(), '{title}')]/ancestor::a")
#         link_element.click()

#         # Wait for the book page to load
#         time.sleep(3)

#         # Scrape the desired information
#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#         gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#         count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#         count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#         date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#         # Scrape the genre
#         genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#         genres = [genre.text.strip() for genre in genre_elements]
#         genre1 = ', '.join(genres) if genres else None

#         # Append scraped data to the self_published list
#         self_published.append({
#             'title': gr_title,
#             'author': gr_author,
#             'avg_rating': avg_rating,
#             'count_rating': count_rating,
#             'count_reviews': count_reviews,
#             'date_published': date_published,
#             'top_genre': genre1
#         })

#     except Exception as e:
#         # Handle the exception here
#         unavailable.append({
#             'title': title,  # Use 'title' variable instead of undefined 'gr_title'
#             'author': None  # Set author as None if not found
#         })


In [171]:
#Still getting some weird results.  Need to search for both title and author before clicking link.  Make a new list of authors
#try a search by title - tried doing a title/author combo earlier and was getting too many errors
#get unique titles, turn into a list

title_author_pairs = list(zip(goodreads_nulls['title'], goodreads_nulls['author']))

In [172]:
# Get unique title_author pairs using set()
unique_title_author_pairs = set(title_author_pairs)

# Convert the unique pairs back to a list
unique_title_author_pairs_list = list(unique_title_author_pairs)

#count
len(unique_title_author_pairs)


1064

In [102]:
#try again using unique title/author pairs


In [103]:
# #try again with both title/author in link_element
# # Set up scraper loop
# self_published = []
# unavailable = []

# # Loop through unique_title_author_pairs
# for title, author in unique_title_author_pairs:
#     try:
#         # Look up the book page URL for the given title
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title)  # Use title as the search query
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find the link with matching title and author
#         link_element = driver.find_element(By.XPATH, f"//a[span[@itemprop='name' and contains(text(), '{title}') and contains(text(), '{author}')]]")
#         link_element.click()

#         # Wait for the book page to load
#         time.sleep(3)

#         # Scrape the desired information
#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#         gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#         count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#         count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#         date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#         # Scrape the genre
#         genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#         genres = [genre.text.strip() for genre in genre_elements]
#         genre1 = ', '.join(genres) if genres else None

#         # Append scraped data to the self_published list
#         self_published.append({
#             'title': gr_title,
#             'author': gr_author,
#             'avg_rating': avg_rating,
#             'count_rating': count_rating,
#             'count_reviews': count_reviews,
#             'date_published': date_published,
#             'top_genre': genre1
#         })

#     except Exception as e:
#         # Handle the exception here
#         unavailable.append({
#             'title': title,
#             'author': author
#         })


In [104]:
# #last one wasn't clicking the correct links. try again looping through search results to compare title/author
# # Set up scraper loop
# self_published = []
# unavailable = []

# # Loop through unique_title_author_pairs
# for title, author in unique_title_author_pairs:
#     try:
#         # Look up the book page URL for the given title
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title)  # Use title as the search query
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find all search results
#         search_results = driver.find_elements(By.XPATH, "//tr[contains(@id, 'bookTitle')]")

#         # Loop through search results to find the matching link
#         for result in search_results:
#             result_title = result.find_element(By.XPATH, ".//a/span[@itemprop='name']").text.strip()
#             result_author = result.find_element(By.XPATH, ".//span[@itemprop='name']").text.strip()

#             # Compare the title and author of the result with the desired values
#             if result_title == title and result_author == author:
#                 link_element = result.find_element(By.XPATH, ".//a")
#                 link_element.click()
#                 break  # Exit the loop if a match is found

#         # Wait for the book page to load
#         time.sleep(3)

#         # Scrape the desired information
#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#         gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#         count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#         count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#         date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#         # Scrape the genre
#         genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#         genres = [genre.text.strip() for genre in genre_elements]
#         genre1 = ', '.join(genres) if genres else None

#         # Append scraped data to the self_published list
#         self_published.append({
#             'title': gr_title,
#             'author': gr_author,
#             'avg_rating': avg_rating,
#             'count_rating': count_rating,
#             'count_reviews': count_reviews,
#             'date_published': date_published,
#             'top_genre': genre1
#         })

#     except Exception as e:
#         # Handle the exception here
#         unavailable.append({
#             'title': title,
#             'author': author
#         })


In [105]:
# #last one still wasn't clicking anything, just searching all the names
# #try again using find_element_by_link_text

# # Set up scraper loop
# self_published = []
# unavailable = []

# # Loop through unique_title_author_pairs
# for title, author in unique_title_author_pairs:
#     try:
#         # Look up the book page URL for the given title
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title)  # Use title as the search query
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find the link with matching title and author
#         link_text = f"{title} ({author})"
#         link_element = driver.find_element_by_link_text(link_text)
#         link_element.click()

#         # Wait for the book page to load
#         time.sleep(3)

#         # Scrape the desired information
#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#         gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#         count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#         count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#         date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#         # Scrape the genre
#         genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#         genres = [genre.text.strip() for genre in genre_elements]
#         genre1 = ', '.join(genres) if genres else None

#         # Append scraped data to the self_published list
#         self_published.append({
#             'title': gr_title,
#             'author': gr_author,
#             'avg_rating': avg_rating,
#             'count_rating': count_rating,
#             'count_reviews': count_reviews,
#             'date_published': date_published,
#             'top_genre': genre1
#         })

#     except Exception as e:
#         # Handle the exception here
#         unavailable.append({
#             'title': title,
#             'author': author
#         })


In [106]:
# #try again clicking on cover art
# # Set up scraper loop
# self_published = []
# unavailable = []

# # Loop through unique_title_author_pairs
# for title, author in unique_title_author_pairs:
#     try:
#         # Look up the book page URL for the given title
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title)  # Use title as the search query
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find the cover art element with a partial match for the alt attribute value
#         cover_element = driver.find_element(By.XPATH, f"//img[contains(@alt, '{title}') and contains(@alt, '{author}')]")

#         # Click on the cover art element
#         cover_element.click()

#         # Wait for the book page to load
#         time.sleep(3)

#         # Scrape the desired information
#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#         gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#         count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#         count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#         date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#         # Scrape the genre
#         genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#         genres = [genre.text.strip() for genre in genre_elements]
#         genre1 = ', '.join(genres) if genres else None

#         # Append scraped data to the self_published list
#         self_published.append({
#             'title': gr_title,
#             'author': gr_author,
#             'avg_rating': avg_rating,
#             'count_rating': count_rating,
#             'count_reviews': count_reviews,
#             'date_published': date_published,
#             'top_genre': genre1
#         })

#     except Exception as e:
#         # Handle the exception here
#         unavailable.append({
#             'title': title,
#             'author': author
#         })


In [107]:
# #same problem, try find_element_by_partial_link_text method
# # Set up scraper loop
# self_published = []
# unavailable = []

# # Loop through unique_title_author_pairs
# for title, author in unique_title_author_pairs:
#     try:
#         # Look up the book page URL for the given title
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title)  # Use title as the search query
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find the link element with partial text matching the title and author
#         link_element = driver.find_element(By.PARTIAL_LINK_TEXT, f"{title} ({author}")

#         # Click on the link element
#         link_element.click()

#         # Wait for the book page to load
#         time.sleep(3)

#         # Scrape the desired information
#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#         gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#         count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#         count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#         date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#         # Scrape the genre
#         genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#         genres = [genre.text.strip() for genre in genre_elements]
#         genre1 = ', '.join(genres) if genres else None

#         # Append scraped data to the self_published list
#         self_published.append({
#             'title': gr_title,
#             'author': gr_author,
#             'avg_rating': avg_rating,
#             'count_rating': count_rating,
#             'count_reviews': count_reviews,
#             'date_published': date_published,
#             'top_genre': genre1
#         })

#     except Exception as e:
#         # Handle the exception here
#         unavailable.append({
#             'title': title,
#             'author': author
#         })


In [108]:
# #nope same problem.  Try finding all the link elements on the page and iterate through to find a match based on both title and author
# # Set up scraper loop
# self_published = []
# unavailable = []

# # Loop through unique_title_author_pairs
# for title, author in unique_title_author_pairs:
#     try:
#         # Look up the book page URL for the given title
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title)  # Use title as the search query
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find all the link elements on the search results page
#         link_elements = driver.find_elements(By.TAG_NAME, "a")

#         # Iterate through the link elements
#         for link_element in link_elements:
#             # Check if the link element text contains both the title and author
#             if title in link_element.text and author in link_element.text:
#                 # Click on the link element
#                 link_element.click()
#                 break

#         # Wait for the book page to load
#         time.sleep(3)

#         # Scrape the desired information
#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#         gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#         count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#         count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#         date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#         # Scrape the genre
#         genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#         genres = [genre.text.strip() for genre in genre_elements]
#         genre1 = ', '.join(genres) if genres else None

#         # Append scraped data to the self_published list
#         self_published.append({
#             'title': gr_title,
#             'author': gr_author,
#             'avg_rating': avg_rating,
#             'count_rating': count_rating,
#             'count_reviews': count_reviews,
#             'date_published': date_published,
#             'top_genre': genre1
#         })

#     except Exception as e:
#         # Handle the exception here
#         unavailable.append({
#             'title': title,
#             'author': author
#         })


In [109]:
# #didn't work because it was looking for both title/author when only title is in that link elem3ent.  
# #try again to search for title only in link_element text and then check if author is present on the book page

# # Set up scraper loop
# self_published = []
# unavailable = []

# # Loop through unique_title_author_pairs
# for title, author in unique_title_author_pairs:
#     try:
#         # Look up the book page URL for the given title
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title)  # Use title as the search query
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find all the link elements on the search results page
#         link_elements = driver.find_elements(By.TAG_NAME, "a")

#         # Iterate through the link elements
#         for link_element in link_elements:
#             # Check if the title is present in the link element text
#             if title in link_element.text:
#                 # Click on the link element
#                 link_element.click()
#                 break

#         # Wait for the book page to load
#         time.sleep(3)

#         # Check if the author is present on the book page
#         author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author}')]")
#         if author_element:
#             # Scrape the desired information
#             soup = BeautifulSoup(driver.page_source, 'html.parser')
#             gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#             gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#             avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#             count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#             count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#             date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#             # Scrape the genre
#             genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#             genres = [genre.text.strip() for genre in genre_elements]
#             genre1 = ', '.join(genres) if genres else None

#             # Append scraped data to the self_published list
#             self_published.append({
#                 'title': gr_title,
#                 'author': gr_author,
#                 'avg_rating': avg_rating,
#                 'count_rating': count_rating,
#                 'count_reviews': count_reviews,
#                 'date_published': date_published,
#                 'top_genre': genre1
#             })
#         else:
#             # Handle the case when the author is not found on the book page
#             unavailable.append({
#                 'title': title,
#                 'author': author
#             })

#     except Exception as e:
#         # Handle the exception here
#         unavailable.append({
#             'title': title,
#             'author': author
#         })


In [110]:
# len(self_published)

In [111]:
# len(unavailable)

In [112]:
# unavailable

In [113]:
# self_published

In [114]:
# #so a lot of these are still being flagged as unavailable, but I see them on goodreads.  Let's try using lower and in

# self_published = []
# unavailable = []

# # Convert unique_title_author_pairs to a list
# unique_pairs_list = list(unique_title_author_pairs)

# # Loop through the pairs using a counter variable
# counter = 0
# for title, author in unique_pairs_list:
#     if counter >= 10:
#         break
    
#     try:
#         # Look up the book page URL for the given title
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title)  # Use title as the search query
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find all the link elements on the search results page
#         link_elements = driver.find_elements(By.TAG_NAME, "a")

#         # Iterate through the link elements
#         for link_element in link_elements:
#             # Check if the title is present in the link element text
#             if title in link_element.text:
#                 # Click on the link element
#                 link_element.click()
#                 break

#         # Wait for the book page to load
#         time.sleep(3)

#         # Check if the author is present on the book page
#         author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author}')]")
#         if author_element:
#             # Scrape the desired information
#             soup = BeautifulSoup(driver.page_source, 'html.parser')
#             gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#             gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#             avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#             count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#             count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#             date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#             # Scrape the genre
#             genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#             genres = [genre.text.strip() for genre in genre_elements]
#             genre1 = ', '.join(genres) if genres else None

#             # Append scraped data to the self_published list
#             self_published.append({
#                 'title': gr_title,
#                 'author': gr_author,
#                 'avg_rating': avg_rating,
#                 'count_rating': count_rating,
#                 'count_reviews': count_reviews,
#                 'date_published': date_published,
#                 'top_genre': genre1
#             })
#         else:
#             # Handle the case when the author is not found on the book page
#             unavailable.append({
#                 'title': title,
#                 'author': author
#             })

#         counter += 1  # Increment the counter after each iteration

#     except Exception as e:
#         # Handle the exception here
#         unavailable.append({
#             'title': title,
#             'author': author
#         })



In [115]:
# len(self_published)

In [116]:
# len(unavailable)

In [117]:
# unavailable

In [118]:
#still having issues with titles with parentheses.  try to use reg expression to remove content within parentheses from link element text before comparing to modified title


In [119]:
# len(self_published)

In [120]:
# len(unavailable)

In [121]:
# unavailable

In [122]:
# #still getting so many unavailable.  try adding more breaks?
# import re

# self_published = []
# unavailable = []

# # Convert unique_title_author_pairs to a list
# unique_pairs_list = list(unique_title_author_pairs)

# # Loop through the first 10 pairs
# for title, author in unique_pairs_list[:10]:
#     try:
#         # Remove content within parentheses from the title
#         title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()

#         # Look up the book page URL for the given title
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title_without_parentheses)  # Use modified title as the search query
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find all the link elements on the search results page
#         link_elements = driver.find_elements(By.TAG_NAME, "a")

#         # Iterate through the link elements
#         for link_element in link_elements:
#             # Remove content within parentheses from the link element text
#             link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

#             # Check if the modified title is present in the modified link element text
#             if title_without_parentheses.lower() in link_text_without_parentheses.lower():
#                 # Click on the link element
#                 link_element.click()
#                 break

#         # Wait for the book page to load
#         time.sleep(3)

#         # Check if the author is present on the book page
#         author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author}')]")
#         if author_element:
#             # Scrape the desired information
#             soup = BeautifulSoup(driver.page_source, 'html.parser')
#             gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#             gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#             avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#             count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#             count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#             date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#             # Scrape the genre
#             genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#             genres = [genre.text.strip() for genre in genre_elements]
#             genre1 = ', '.join(genres) if genres else None

#             # Append scraped data to the self_published list
#             self_published.append({
#                 'title': gr_title,
#                 'author': gr_author,
#                 'avg_rating': avg_rating,
#                 'count_rating': count_rating,
#                 'count_reviews': count_reviews,
#                 'date_published': date_published,
#                 'top_genre': genre1
#             })
#         else:
#             # Handle the case when the author is not found on the book page
#             unavailable.append({
#                 'title': title,
#                 'author': author
#             })

#     except:
#         # Handle any exceptions that occur during the scraping process
#         unavailable.append({
#             'title': title,
#             'author': author
#         })


In [123]:
# #maybe we should only check for authors last name on page, to avoid issues with punctuation and mutiple authors
# # Check if the author's last name is present on the book page
# self_published = []
# unavailable = []

# # Convert unique_title_author_pairs to a list
# unique_pairs_list = list(unique_title_author_pairs)

# # Loop through the first 10 pairs
# for title, author in unique_pairs_list:
#     try:
#         # Remove content within parentheses from the title
#         title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()

#         # Look up the book page URL for the given title
#         search_book = driver.find_element(By.NAME, "q")
#         search_book.clear()  # Clear any previous search
#         search_book.send_keys(title_without_parentheses)  # Use modified title as the search query
#         search_book.submit()

#         # Wait for search results to load
#         time.sleep(3)

#         # Find all the link elements on the search results page
#         link_elements = driver.find_elements(By.TAG_NAME, "a")

#         # Iterate through the link elements
#         for link_element in link_elements:
#             # Remove content within parentheses from the link element text
#             link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

#             # Check if the modified title is present in the modified link element text
#             if title_without_parentheses.lower() in link_text_without_parentheses.lower():
#                 # Click on the link element
#                 link_element.click()
#                 break

#         # Wait for the book page to load
#         time.sleep(3)

#         # Check if the author is present on the book page
#         author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
#         if author_element:
#             # Scrape the desired information
#             soup = BeautifulSoup(driver.page_source, 'html.parser')
#             gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#             gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#             avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#             count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#             count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#             date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#             # Scrape the genre
#             genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#             genres = [genre.text.strip() for genre in genre_elements]
#             genre1 = ', '.join(genres) if genres else None

#             # Append scraped data to the self_published list
#             self_published.append({
#                 'title': gr_title,
#                 'author': gr_author,
#                 'avg_rating': avg_rating,
#                 'count_rating': count_rating,
#                 'count_reviews': count_reviews,
#                 'date_published': date_published,
#                 'top_genre': genre1
#             })
#         else:
#             # Handle the case when the author is not found on the book page
#             unavailable.append({
#                 'title': title,
#                 'author': author
#             })

#     except:
#         # Handle any exceptions that occur during the scraping process
#         unavailable.append({
#             'title': title,
#             'author': author
#         })


In [124]:
# len(self_published)

In [125]:
# len(unavailable)

In [126]:
# unavailable

In [173]:
# #keep getting 502 bad gateway errors.  Add a retry_count variable, set max number of retried to 3.  
# #Also try wrapping scraping code in 'while' loop that runs until either scraping is successful, author is not found or max number of retries is reached
# #Inside the while loop, add delay of 5 seconds before retrying

# import re
# import time
# import string

# final_found = []
# final_unavailable = []

# # Convert unique_title_author_pairs to a list
# unique_pairs_list = list(unique_title_author_pairs)

# # Loop through the pairs
# for title, author in unique_pairs_list[:10]:
#     retry_count = 0
#     max_retries = 3
    
#     while retry_count < max_retries:
#         try:
#             # Remove content within parentheses from the title
#             title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()
#             title_without_punctuation = title_without_parentheses.translate(str.maketrans('', '', string.punctuation))

#             # Look up the book page URL for the given title
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(title_without_punctuation)  # Use modified title as the search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(3)

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             # Iterate through the link elements
#             for link_element in link_elements:
#                 # Remove content within parentheses from the link element text
#                 link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

#                 # Check if the modified title is present in the modified link element text
#                 if title_without_parentheses.lower() in link_text_without_parentheses.lower():
#                     # Click on the link element
#                     link_element.click()
#                     break

#             # Wait for the book page to load
#             time.sleep(3)

#             # Check if the author's last name is present on the book page
#             author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
#             if author_element:
#                 # Scrape the desired information
#                 soup = BeautifulSoup(driver.page_source, 'html.parser')
#                 gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#                 gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                 avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                 count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#                 count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#                 date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#                 # Scrape the genre
#                 genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#                 genres = [genre.text.strip() for genre in genre_elements]
#                 genre1 = ', '.join(genres) if genres else None

#                 # Append scraped data to the self_published list
#                 final_found.append({
#                     'title': gr_title,
#                     'author': gr_author,
#                     'avg_rating': avg_rating,
#                     'count_rating': count_rating,
#                     'count_reviews': count_reviews,
#                     'date_published': date_published,
#                     'top_genre': genre1
#                 })
#                 break  # Exit the retry loop if scraping is successful

#             else:
#                 # Handle the case when the author is not found on the book page
#                 final_unavailable.append({
#                     'title': title,
#                     'author': author
#                 })
#                 break  # Exit the retry loop if author is not found

#         except Exception as e:
#             print(f"Error occurred: {str(e)}")
#             retry_count += 1
#             print(f"Retrying ({retry_count}/{max_retries})...")
#             time.sleep(5)  # Delay before retrying the scraping operation

#     if retry_count == max_retries:
#         # Handling the case when the maximum number of retries is reached
#         final_unavailable.append({
#             'title': title,
#             'author': author
#         })


In [128]:
# #if needed, here is one that will hit the reload button after a 502 bad gateway, and also will try to click a different title link if the first one is wrong
# Also, try taking out : and - from titles
##maybe try adding author last name back into search?


# import re
# import time

# self_published = []
# unavailable = []

# # Convert unique_title_author_pairs to a list
# unique_pairs_list = list(unique_title_author_pairs)

# # Loop through the pairs
# for title, author in unique_pairs_list:
#     retry_count = 0
#     max_retries = 3
    
#     while retry_count < max_retries:
#         try:
#             # Remove content within parentheses from the title
#             title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()

#             # Look up the book page URL for the given title
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(title_without_parentheses)  # Use modified title as the search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(3)

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             clicked = False  # Flag to track if a successful click has been made

#             # Iterate through the link elements
#             for link_element in link_elements:
#                 # Remove content within parentheses from the link element text
#                 link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

#                 # Check if the modified title is present in the modified link element text
#                 if title_without_parentheses.lower() in link_text_without_parentheses.lower():
#                     # Click on the link element
#                     link_element.click()
#                     clicked = True
#                     break

#             if clicked:
#                 # Wait for the book page to load
#                 time.sleep(3)

#                 # Check if the author's last name is present on the book page
#                 author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
#                 if author_element:
#                     # Scrape the desired information
#                     soup = BeautifulSoup(driver.page_source, 'html.parser')
#                     gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#                     gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                     avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                     count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#                     count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#                     date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#                     # Scrape the genre
#                     genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#                     genres = [genre.text.strip() for genre in genre_elements]
#                     genre1 = ', '.join(genres) if genres else None

#                     # Append scraped data to the self_published list
#                     self_published.append((gr_title, gr_author, avg_rating, count_rating, count_reviews, date_published, genre1))

#                     break  # Exit the while loop if successful

#         except (NoSuchElementException, StaleElementReferenceException, TimeoutException) as e:
#             print(f"An exception occurred: {str(e)}")
#             print("Retrying...")
#             retry_count += 1
#             time.sleep(2)  # Delay before retrying

#         except Exception as e:
#             print(f"An unexpected exception occurred: {str(e)}")
#             break  # Exit the while loop if an unexpected exception occurs

#     if retry_count == max_retries:
#         print(f"Max retries reached for title: {title}, author: {author}")
#         unavailable.append((title, author))


In [129]:
# len(self_published)

In [130]:
# len(unavailable)

In [131]:
# unavailable

In [132]:
# self_published = pd.DataFrame(self_published)

In [133]:
# self_published.head()

In [134]:
# #Fix date published column
# self_published['date_published'] = self_published['date_published'].str.replace('First published ', '').str.replace('Published ', '')
# self_published['date_published'] = pd.to_datetime(self_published['date_published'], errors='coerce')
# self_published.head()

In [135]:
# #Fix genre column
# self_published['top_genre'] = self_published['top_genre'].str.replace('Genres, ', '')


In [136]:
# #Split column into list of genres
# genre_list = self_published['top_genre'].str.split(', ')

In [137]:
# self_published['genre1'] = genre_list.str[0]
# self_published['genre2'] = genre_list.str[1]
# self_published['genre3'] = genre_list.str[2]

In [138]:
# self_published.head()

In [139]:
# #Try again with the unavailable list
# #Take out all punctuation from title
# #Search by title + author last name
# import re
# import time

# found = []
# still_unavailable = []

# # Loop through the unavailable list
# for item in unavailable:
#     title = item['title']
#     author = item['author']
#     retry_count = 0
#     max_retries = 3

#     while retry_count < max_retries:
#         try:
#             # Remove all punctuation from the title
#             title_without_punctuation = re.sub(r'[^\w\s]', '', title).strip()

#             # Extract the author's last name
#             author_last_name = author.split()[-1]

#             # Look up the book page URL for the given title and author's last name
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(title_without_punctuation + ' ' + author_last_name)  # Use modified title and author's last name as the search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(3)

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             # Iterate through the link elements
#             link_clicked = False
#             for link_element in link_elements:
#                 # Remove all punctuation from the link element text
#                 link_text_without_punctuation = re.sub(r'[^\w\s]', '', link_element.text).strip()

#                 # Check if the modified title without punctuation is present in the modified link element text
#                 if title_without_punctuation.lower() in link_text_without_punctuation.lower():
#                     # Click on the link element
#                     link_element.click()
#                     link_clicked = True
#                     break

#             if link_clicked:
#                 # Wait for the book page to load
#                 time.sleep(3)

#                 # Check if the author's last name is present on the book page
#                 author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author_last_name}')]")
#                 if author_element:
#                     # Scrape the desired information
#                     soup = BeautifulSoup(driver.page_source, 'html.parser')
#                     gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#                     gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                     avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                     count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#                     count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#                     date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#                     # Scrape the genre
#                     genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#                     genres = [genre.text.strip() for genre in genre_elements]
#                     genre1 = ', '.join(genres) if genres else None

#                     # Append scraped data to the found list
#                     found.append({
#                         'title': gr_title,
#                         'author': gr_author,
#                         'avg_rating': avg_rating,
#                         'count_rating': count_rating,
#                         'count_reviews': count_reviews,
#                         'date_published': date_published,
#                         'top_genre': genre1
#                     })
#                     break  # Exit the retry loop if scraping is successful

#             retry_count += 1
#             print(f"Retrying ({retry_count}/{max_retries})...")
#             time.sleep(5)  # Delay before retrying the scraping operation

#         except Exception as e:
#             print(f"Error occurred: {str(e)}")
#             retry_count += 1
#             print(f"Retrying ({retry_count}/{max_retries})...")
#             time.sleep(5)  # Delay before retrying the scraping operation

#     if retry_count == max_retries:
#         # Handling the case when the maximum number of retries is reached
#         still_unavailable.append({
#             'title': title,
#             'author': author
#         })





In [140]:
# #still running into issues with Summary books.  Here's an idea for eliminating those
# # Iterate through the link elements
# link_clicked = False
# for link_element in link_elements:
#     # Remove all punctuation from the link element text
#     link_text_without_punctuation = re.sub(r'[^\w\s]', '', link_element.text).strip()

#     # Check if the modified title without punctuation is present in the modified link element text
#     if title_without_punctuation.lower() in link_text_without_punctuation.lower() and 'summary' | 'study guide' | 'synopsis' not in link_text_without_punctuation.lower():
#         # Click on the link element
#         link_element.click()
#         link_clicked = True
#         break

# if link_clicked:
#     # Wait for the book page to load
#     time.sleep(3)

#     # Check if the title includes "Summary"
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None

#     if gr_title and 'summary' not in gr_title.lower():
#         # Check if the author's last name is present on the book page
#         author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author_last_name}')]")
#         if author_element:
#             # Scrape the desired information
#             gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#             avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#             count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#             count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#             date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#             # Scrape the genre
#             genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#             genres = [genre.text.strip() for genre in genre_elements]
#             genre1 = ', '.join(genres) if genres else None

#             # Append scraped data to the found list
#             found.append({
#                 'title': gr_title,
#                 'author': gr_author,
#                 'avg_rating': avg_rating,
#                 'count_rating': count_rating,
#                 'count_reviews': count_reviews,
#                 'date_published': date_published,
#                 'top_genre': genre1
#             })
#             break  # Exit the retry loop if scraping is successful

# retry_count += 1
# print(f"Retrying ({retry_count}/{max_retries})...")
# time.sleep(5)  # Delay before retrying the scraping operation


In [141]:
# len(found)

In [142]:
# len(still_unavailable)

In [143]:
# found

In [144]:
# still_unavailable

In [145]:
# #still running into issues with Summary books.  Here's an idea for eliminating those
# # Iterate through the link elements
# import re
# import time

# final_found = []
# final_unavailable = []

# # Loop through the unavailable list
# for item in still_unavailable[:10]:
#     title = item['title']
#     author = item['author']
#     retry_count = 0
#     max_retries = 3

#     while retry_count < max_retries:
#         try:
#             # Remove all punctuation from the title
#             title_without_punctuation = re.sub(r'[^\w\s]', '', title).strip()

#             # Extract the author's last name
#             author_last_name = author.split()[-1]

#             # Look up the book page URL for the given title and author's last name
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(title_without_punctuation + ' ' + author_last_name)  # Use modified title and author's last name as the search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(3)

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             link_clicked = False
#             for link_element in link_elements:
#                 # Remove all punctuation from the link element text
#                 link_text_without_punctuation = re.sub(r'[^\w\s]', '', link_element.text).strip()

#                 # Check if the modified title without punctuation is present in the modified link element text
#                 if title_without_punctuation.lower() in link_text_without_punctuation.lower() and not any(word in link_text_without_punctuation.lower() for word in ['summary', 'study guide', 'synopsis']):
#                     # Click on the link element
#                     link_element.click()
#                     link_clicked = True
#                     break

#             if link_clicked:
#                 # Wait for the book page to load
#                 time.sleep(3)

#                 # Check if the title includes "Summary"
#                 soup = BeautifulSoup(driver.page_source, 'html.parser')
#                 gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None

#                 if gr_title and 'summary' not in gr_title.lower():
#                     # Check if the author's last name is present on the book page
#                     author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author_last_name}')]")
#                     if author_element:
#                         # Scrape the desired information
#                         gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                         count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#                         count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#                         date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#                         # Scrape the genre
#                         genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#                         genres = [genre.text.strip() for genre in genre_elements]
#                         genre1 = ', '.join(genres) if genres else None

#                         # Append scraped data to the found list
#                         found.append({
#                             'title': gr_title,
#                             'author': gr_author,
#                             'avg_rating': avg_rating,
#                             'count_rating': count_rating,
#                             'count_reviews': count_reviews,
#                             'date_published': date_published,
#                             'top_genre': genre1
#                         })
#                         break  # Exit the retry loop if scraping is successful

#             retry_count += 1
#             print(f"Retrying ({retry_count}/{max_retries})...")
#             time.sleep(5)  # Delay before retrying the scraping operation

#         except Exception as e:
#             print(f"Error occurred: {e}")
#             retry_count += 1
#             print(f"Retrying ({retry_count}/{max_retries})...")
#             time.sleep(5)  # Delay before retrying the scraping operation


In [146]:
# len(final_found)

In [147]:
# len(final_unavailable)

In [163]:
unavailable = bestsellers[bestsellers['title_y'].isnull()]
unavailable.head()

,title,description,author,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,week,...,title_y,author_y,avg_rating,count_rating,count_reviews,date_published,top_genre,genre1,genre2,genre3
0,Full Package,A man shares a cramped apartment with his frie...,Lauren Blakely,Lauren Blakely,A00B01MT5HMRV,None,1,0,1,2017-01-25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Shelter For Adeline,A fireman must keep his overprotective nature ...,Susan Stoker,Stoker Aces Production,A00B01MF62CN8,None,6,0,1,2017-01-25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,Until Ashlyn,An impulsive Vegas wedding turns out to have s...,Aurora Rose Reynolds,Aurora Rose Reynolds,A00B01LYEOR3I,None,8,0,1,2017-01-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,Melt,"In this fourth book in the series, the oldest ...",Helen Hardt,Waterhouse,A00B01HFPDRI4,None,1,0,1,2017-01-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Craving,A jilted bride decamps to a Colorado ranch whe...,Helen Hardt,Waterhouse,A00B01FWPZQRY,None,10,0,2,2017-01-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [164]:
len(unavailable)

6843

In [165]:
#get distinct title/author pairs from unavailable list

import re

titles = bestsellers['title']
authors = bestsellers['author']

# Extract unique titles without punctuation and author's last name
unique_title_author_pairs = set(zip(titles, authors))
unique_pairs_list = list(unique_title_author_pairs)

title_with_last_name= [re.sub(r'[^\w\s]', '', str(title)).strip() + ' ' + str(author).split()[-1] for title, author in unique_pairs_list]


In [166]:
len(unique_title_author_pairs)

4188

In [152]:
# #fixing to make sure retries click on different elements

# import re
# import time

# final_found = []
# final_unavailable = []

# # Function to find and return the link element for the given title and required keywords
# def find_link_element(title, required_keywords):
#     # Find all the link elements on the search results page
#     link_elements = driver.find_elements(By.TAG_NAME, "a")

#     for link_element in link_elements:
#         # Remove all punctuation from the link element text
#         link_text_without_punctuation = re.sub(r'[^\w\s]', '', link_element.text).strip()

#         # Check if the modified title without punctuation is present in the modified link element text
#         if title.lower() in link_text_without_punctuation.lower() and all(keyword in link_text_without_punctuation.lower() for keyword in required_keywords):
#             return link_element

#     return None

# # Loop through the unavailable list
# for item in title_with_last_name[:10]:
#     title_with_author = item.split()
#     title = ' '.join(title_with_author[:-1])
#     author_last_name = title_with_author[-1]

#     retry_count = 0
#     max_retries = 3
#     link_clicked = False

#     while retry_count < max_retries and not link_clicked:
#         try:
#             # Remove all punctuation from the title
#             title_without_punctuation = re.sub(r'[^\w\s]', '', title).strip()

#             # Look up the book page URL for the given title and author's last name
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(title_without_punctuation + ' ' + author_last_name)  # Use modified title and author's last name as the search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(3)

#             # Find the link element for the desired book
#             link_element = find_link_element(title_without_punctuation, ['summary', 'synopsis', 'study', 'review'])

#             if link_element:
#                 # Click on the link element
#                 link_element.click()
#                 link_clicked = True

#             if link_clicked:
#                 # Wait for the book page to load
#                 time.sleep(3)

#                 # Check if the title includes "Summary"
#                 soup = BeautifulSoup(driver.page_source, 'html.parser')
#                 gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None

#                 if gr_title and 'summary' not in gr_title.lower():
#                     # Check if the author's last name is present on the book page
#                     author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author_last_name}')]")
#                     if author_element:
#                         # Scrape the desired information
#                         gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                         count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split
#                         # Append to final_found list
#                         final_found.append({
#                             'Title': gr_title,
#                             'Author': gr_author,
#                             'Average Rating': avg_rating,
#                             'Count Rating': count_rating
#                         })
#                     else:
#                         # Author's last name not found on the book page
#                         final_unavailable.append(item)
#                 else:
#                     # "Summary" keyword found in the title
#                     final_unavailable.append(item)
#             else:
#                 # Link element not found
#                 final_unavailable.append(item)
#         except:
#             # Retry search in case of any error
#             retry_count += 1
#             continue

#         # Increment the retry count
#         retry_count += 1


In [153]:
# len(final_found)

0

In [154]:
# len(final_unavailable)

0

In [155]:
# #last round didn't click on any element, just searched.  Also put everything into the final_unavailable list 3 times

# import re
# import time

# final_found = []
# final_unavailable = []

# # Function to find and return the link element for the given title and required keywords
# def find_link_element(title, required_keywords):
#     # Find all the link elements on the search results page
#     link_elements = driver.find_elements(By.TAG_NAME, "a")

#     for link_element in link_elements:
#         # Remove all punctuation from the link element text
#         link_text_without_punctuation = re.sub(r'[^\w\s]', '', link_element.text).strip()

#         # Check if the modified title without punctuation is present in the modified link element text
#         if title.lower() in link_text_without_punctuation.lower() and all(keyword in link_text_without_punctuation.lower() for keyword in required_keywords):
#             return link_element

#     return None

# # Loop through the unavailable list
# for item in title_with_last_name[:10]:
#     title_with_author = item.split()
#     title = ' '.join(title_with_author[:-1])
#     author_last_name = title_with_author[-1]

#     retry_count = 0
#     max_retries = 3
#     link_clicked = False

#     while retry_count < max_retries and not link_clicked:
#         try:
#             # Remove all punctuation from the title
#             title_without_punctuation = re.sub(r'[^\w\s]', '', title).strip()

#             # Look up the book page URL for the given title and author's last name
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(title_without_punctuation + ' ' + author_last_name)  # Use modified title and author's last name as the search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(3)

#             # Find the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             for link_element in link_elements:
#                 # Check if the link element text contains the required keywords
#                 link_text = link_element.text.strip().lower()
#                 if all(keyword in link_text for keyword in ['summary', 'synopsis', 'study', 'review']):
#                     # Click on the link element
#                     link_element.click()
#                     link_clicked = True
#                     break

#             if link_clicked:
#                 # Wait for the book page to load
#                 time.sleep(3)

#                 # Check if the title includes "Summary"
#                 soup = BeautifulSoup(driver.page_source, 'html.parser')
#                 gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None

#                 if gr_title and 'summary' not in gr_title.lower():
#                     # Check if the author's last name is present on the book page
#                     author_element = soup.find('span', {'class': 'ContributorLink__name'}, text=re.compile(rf"{author_last_name}", re.I))
#                     if author_element:
#                         # Scrape the desired information
#                         gr_author = author_element.text.strip() if author_element else None
#                         avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                         count_rating = soup.find('span', {'data-reactid': '21'}).text.strip() if soup.find('span', {'data-reactid': '21'}) else None
#                         reviews = soup.find('a', {'class': 'Button Button--text Button--small Button--link Button--white'}).text.strip() if soup.find('a', {'class': 'Button Button--text Button--small Button--link Button--white'}) else None

#                         # Append the scraped information to the final_found list
#                         final_found.append({
#                             'Title': gr_title,
#                             'Author': gr_author,
#                             'Average Rating': avg_rating,
#                             'Rating Count': count_rating,
#                             'Reviews': reviews
#                         })
#                     else:
#                         # Author's last name not found on the book page
#                         final_unavailable.append(item)
#                 else:
#                     # "Summary" keyword found in the title
#                     final_unavailable.append(item)
#             else:
#                 # Link element not found
#                 final_unavailable.append(item)
#         except:
#             # Retry search in case of any error
#             retry_count += 1
#             continue

#         # Increment the retry count
#         retry_count += 1

# # Print the final results
# print("Final Found:")
# for item in final_found:
#     print(item)

# print("Final Unavailable:")
# for item in final_unavailable:
#     print(item)



In [156]:
#that one was a mess.  I'm getting lost in the weeds so going to try something new



In [157]:
# import re
# import time

# final_found = []
# final_unavailable = []

# # Function to clean the title and author values
# def clean_text(text):
#     # Remove punctuation and trailing spaces
#     text = re.sub(r'[^\w\s]', '', text).strip()
#     # Remove anything inside parentheses
#     text = re.sub(r'\(.*?\)', '', text).strip()
#     return text

# # Loop through the unique title/author pairs
# for title, author in unique_pairs_list:
#     clean_title = clean_text(title)
#     author_last_name = clean_text(author).split()[-1]

#     retry_count = 0
#     max_retries = 3
#     link_clicked = False

#     while retry_count < max_retries and not link_clicked:
#         try:
#             # Clear the search bar and enter the clean title and author
#             search_input = driver.find_element(By.NAME, "q")
#             search_input.clear()
#             search_input.send_keys(clean_title + ' ' + author_last_name)
#             search_input.submit()

#             time.sleep(2)  # Add a delay to let the page load

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             for link_element in link_elements:
#                 link_text = link_element.text

#                 # Check if the link contains the clean title and doesn't contain any excluded keywords
#                 if clean_title.lower() in link_text.lower() and not any(keyword in link_text.lower() for keyword in ['summary', 'review', 'guide', 'synopsis', 'study']):
#                     link_element.click()
#                     link_clicked = True
#                     break

#             if not link_clicked:
#                 # Retry by incrementing the retry count
#                 retry_count += 1

#         except Exception as e:
#             print(f"Error: {e}")

#     if link_clicked:
#         # Check if the author's last name matches the contributor element on the book page
#         contributor_element = driver.find_element(By.CLASS_NAME, "ContributorLink__name")
#         contributor_name = contributor_element.text
#         if author_last_name.lower() in contributor_name.lower():
#             # Scrape the page as before and append to the final_found list
#             final_found.append((title, author))
#         else:
#             # Retry by incrementing the retry count
#             retry_count += 1

#     if retry_count >= max_retries or not link_clicked:
#         # Append to the final_unavailable list
#         final_unavailable.append((title, author))


In [158]:
# len(final_found)

In [159]:
# len(final_unavailable)

In [170]:
# #That one worked better but I want to try having it check for the author as part of the initial search, as well as making sure there are at least 10 reviews in the review count to avoid weird one-offs

# from selenium.webdriver.common.by import By
# from selenium.webdriver.support import expected_conditions as EC

# final_found = []
# final_unavailable = []

# # Function to clean the title and author values
# def clean_text(text):
#     cleaned_text = re.sub(r'[^\w\s]', '', text).strip()
#     cleaned_text = re.sub(r'\([^()]*\)', '', cleaned_text).strip()
#     return cleaned_text

# # Loop through the unique title/author pairs
# for title, author in unique_pairs_list:
#     # Clean the title and author values
#     cleaned_title = clean_text(title)
#     author_last_name = author.split()[-1]

#     # Identify the search box element and assign it to the `search_box` variable
#     search_box = driver.find_element(By.XPATH, "/html/body/div[2]/div/header/div[2]/div/div[2]/form/input[1]")

#     #Pause for 2 seconds to find
#     time.sleep(2)

#     # Perform the search
#     search_box.clear()
#     search_box.send_keys(cleaned_title + ' ' + author_last_name)
#     search_box.send_keys(Keys.RETURN)

#     # Check the author and review count for each title element on the search page
#     title_elements = driver.find_elements(By.CSS_SELECTOR, "span[itemprop='name']")
#     for element in title_elements:
#         title_element = element.get_attribute('innerHTML')
#         author_element = element.find_element(By.XPATH, "./td[2]/span[2]/div/a/span").get_attribute('innerHTML')
#         review_count_element = element.find_element(By.XPATH, "./td[2]/div[1]/span/span").get_attribute('innerHTML')

#         # Clean the author and review count values
#         cleaned_author = clean_text(author_element)
#         review_count = int(re.findall(r'\d+', review_count_element)[-1])

#         # Check if the author and review count meet the criteria
#         if cleaned_author == author_last_name and review_count >= 10:
#             # Click on the title element
#             element.click()
#             time.sleep(1)

#             # Perform the scraping and append to the appropriate list
#             if scrape_book_page():
#                 final_found.append((title, author))
#             else:
#                 final_unavailable.append((title, author))
#             break

#     # Retry up to 3 times if no matching element is found
#     else:
#         for _ in range(2):
#             next_button = driver.find_element(By.CSS_SELECTOR, "a.next_page")
#             if next_button.is_enabled():
#                 next_button.click()
#                 time.sleep(1)
#                 title_elements = driver.find_elements(By.CSS_SELECTOR, "span[itemprop='name']")
#                 for element in title_elements:
#                     title_element = element.get_attribute('innerHTML')
#                     author_element = element.find_element(By.XPATH, "../following-sibling::span[@itemprop='name']").get_attribute('innerHTML')
#                     review_count_element = element.find_element(By.XPATH, "../following-sibling::span[@class='minirating']").get_attribute('innerHTML')

#                     # Clean the author and review count values
#                     cleaned_author = clean_text(author_element)
#                     review_count = int(re.findall(r'\d+', review_count_element)[-1])

#                     # Check if the author and review count meet the criteria
#                     if cleaned_author == author_last_name and review_count >= 10:
#                         # Click on the title element
#                         element.click()
#                         time.sleep(1)

#                         # Perform the scraping and append to the appropriate list
#                         if scrape_book_page():
#                             final_found.append((title, author))
#                         else:
#                             final_unavailable.append((title, author))
#                         break
#                 else:
#                     continue
#                 break
#         else:
#             final_unavailable.append((title, author))

# # Close the WebDriver
# driver.quit()



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/header/div[2]/div/div[2]/form/input[1]"}
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x011FA813+48355]
	(No symbol) [0x0118C4B1]
	(No symbol) [0x01095358]
	(No symbol) [0x010C09A5]
	(No symbol) [0x010C0B3B]
	(No symbol) [0x010EE232]
	(No symbol) [0x010DA784]
	(No symbol) [0x010EC922]
	(No symbol) [0x010DA536]
	(No symbol) [0x010B82DC]
	(No symbol) [0x010B93DD]
	GetHandleVerifier [0x0145AABD+2539405]
	GetHandleVerifier [0x0149A78F+2800735]
	GetHandleVerifier [0x0149456C+2775612]
	GetHandleVerifier [0x012851E0+616112]
	(No symbol) [0x01195F8C]
	(No symbol) [0x01192328]
	(No symbol) [0x0119240B]
	(No symbol) [0x01184FF7]
	BaseThreadInitThunk [0x76037D59+25]
	RtlInitializeExceptionChain [0x7799B74B+107]
	RtlClearBits [0x7799B6CF+191]


In [174]:
# len(final_found)

10

In [175]:
# final_found

[{'title': 'The Sandman: Act II',
  'author': 'Dirk Maggs',
  'avg_rating': '4.40',
  'count_rating': '7929',
  'count_reviews': '814',
  'date_published': 'First published September 22, 2021',
  'top_genre': 'Genres, Fantasy, Audiobook, Fiction, Horror, Graphic Novels, Comics, Mythology'},
 {'title': 'Country Strong',
  'author': 'Linda Lael Miller',
  'avg_rating': '3.68',
  'count_rating': '2053',
  'count_reviews': '226',
  'date_published': 'First published January 28, 2020',
  'top_genre': 'Genres, Romance, Fiction, Westerns, Contemporary, Contemporary Romance, Adult Fiction, Western Romance'},
 {'title': 'The Soulmate Equation',
  'author': 'Christina Lauren',
  'avg_rating': '4.02',
  'count_rating': '182880',
  'count_reviews': '21110',
  'date_published': 'First published May 18, 2021',
  'top_genre': 'Genres, Romance, Contemporary, Fiction, Contemporary Romance, Audiobook, Adult, Chick Lit'},
 {'title': 'Silverview',
  'author': 'John le Carré',
  'avg_rating': '3.70',
  'co

In [179]:
# #from 173
# #Last one only searched title, need to concatenate with the author's last name
# #Eliminate titles that include 'summary', 'synopsis', 'review', 'study', 'analysis'


# import re
# import string
# import time
# from selenium.common.exceptions import NoSuchElementException

# final_found = []
# final_unavailable = []

# # Convert unique_title_author_pairs to a list
# unique_pairs_list = list(unique_title_author_pairs)

# # Loop through the pairs
# for title, author in unique_pairs_list[:10]:
#     retry_count = 0
#     max_retries = 3
#     found = False
    
#     while retry_count < max_retries and not found:
#         try:
#             # Remove content within parentheses and punctuation from the title
#             title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()
#             title_without_punctuation = title_without_parentheses.translate(str.maketrans('', '', string.punctuation))

#             # Concatenate the title and author's last name
#             search_query = f"{title_without_punctuation} {author.split()[-1]}"

#             # Look up the book page URL for the given title and author
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(search_query)  # Use modified search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(3)

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             # Iterate through the link elements
#             for link_element in link_elements:
#                 # Remove content within parentheses from the link element text
#                 link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

#                 # Check if the modified title is present in the modified link element text
#                 if title_without_parentheses.lower() in link_text_without_parentheses.lower():
#                     # Check if the link text contains any excluded keywords
#                     excluded_keywords = ['summary', 'analysis', 'review', 'synopsis', 'study', 'brief']
#                     if any(keyword in link_text_without_parentheses.lower() for keyword in excluded_keywords):
#                         continue  # Skip this link element if it contains excluded keywords

#                     # Click on the link element
#                     link_element.click()
#                     found = True
#                     break

#             # Wait for the book page to load
#             time.sleep(3)

#             # Check if the author's last name is present on the book page
#             try:
#                 author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
#             except NoSuchElementException:
#                 author_element = None

#             if author_element:
#                 # Scrape the desired information
#                 soup = BeautifulSoup(driver.page_source, 'html.parser')
#                 gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#                 gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                 avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                 count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#                 count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#                 date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#                 # Scrape the genre
#                 genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#                 genres = [genre.text.strip() for genre in genre_elements]
#                 genre1 = ', '.join(genres) if genres else None

#                 # Append scraped data to the final_found list
#                 final_found.append({
#                     'title': gr_title,
#                     'author': gr_author,
#                     'avg_rating': avg_rating,
#                     'count_rating': count_rating,
#                     'count_reviews': count_reviews,
#                     'date_published': date_published,
#                     'top_genre': genre1
#                 })
#                 break  # Exit the retry loop if scraping is successful

#             else:
#                 # Handle the case when the author is not found on the book page
#                 final_unavailable.append({
#                     'title': title,
#                     'author': author
#                 })
#                 break  # Exit the retry loop if author is not found

#         except Exception as e:
#             print(f"Error occurred: {str(e)}")
#             retry_count += 1
#             print(f"Retrying ({retry_count}/{max_retries})...")
#             time.sleep(5)  # Delay before retrying the scraping operation

#     if retry_count == max_retries:
#         # Handling the case when the maximum number of retries is reached
#         final_unavailable.append({
#             'title': title,
#             'author': author
#         })


In [185]:
# #That time worked better, but I want to build in a retry mechanism in case the author isn't found on the book page
# import re
# import string
# import time
# from selenium.common.exceptions import NoSuchElementException

# final_found = []
# final_unavailable = []

# # Convert unique_title_author_pairs to a list
# unique_pairs_list = list(unique_title_author_pairs)

# # Loop through the pairs
# for title, author in unique_pairs_list[20:40]:
#     retry_count = 0
#     max_retries = 3
#     found = False
    
#     while retry_count < max_retries and not found:
#         try:
#             # Remove content within parentheses and punctuation from the title
#             title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()
#             title_without_punctuation = title_without_parentheses.translate(str.maketrans('', '', string.punctuation))

#             # Concatenate the title and author's last name
#             search_query = f"{title_without_punctuation} {author.split()[-1]}"

#             # Look up the book page URL for the given title and author
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(search_query)  # Use modified search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(3)

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             # Iterate through the link elements
#             for link_element in link_elements:
#                 # Remove content within parentheses from the link element text
#                 link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

#                 # Check if the modified title is present in the modified link element text
#                 if title_without_parentheses.lower() in link_text_without_parentheses.lower():
#                     # Check if the link text contains any excluded keywords
#                     excluded_keywords = ['summary', 'analysis', 'review', 'synopsis', 'study']
#                     if any(keyword in link_text_without_parentheses.lower() for keyword in excluded_keywords):
#                         continue  # Skip this link element if it contains excluded keywords

#                     # Click on the link element
#                     link_element.click()
#                     found = True
#                     break

#             # Wait for the book page to load
#             time.sleep(3)

#             # Check if the author's last name is present on the book page
#             try:
#                 author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
#             except NoSuchElementException:
#                 author_element = None

#             if author_element:
#                 soup = BeautifulSoup(driver.page_source, 'html.parser')
#                 gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#                 gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                 avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                 count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#                 count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#                 date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#                 # Scrape the genre
#                 genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#                 genres = [genre.text.strip() for genre in genre_elements]
#                 genre1 = ', '.join(genres) if genres else None

#                 # Append scraped data to the final_found list
#                 final_found.append({
#                     'title': gr_title,
#                     'author': gr_author,
#                     'avg_rating': avg_rating,
#                     'count_rating': count_rating,
#                     'count_reviews': count_reviews,
#                     'date_published': date_published,
#                     'top_genre': genre1
#                 })
#                 break

#         except Exception as e:
#             print(f"Error occurred: {str(e)}")
#             retry_count += 1
#             print(f"Retrying ({retry_count}/{max_retries})...")
#             time.sleep(5)  # Delay before retrying the scraping operation

#     if not found and retry_count == max_retries:
#         # Handling the case when the maximum number of retries is reached
#         final_unavailable.append({
#             'title': title,
#             'author': author
#         })


KeyboardInterrupt: 

In [190]:
# #That worked mostly well, except it got stuck searching the same title/author pair that were not found
# import re
# import string
# import time
# from selenium.common.exceptions import NoSuchElementException
# from selenium.common.exceptions import WebDriverException

# final_found = []
# final_unavailable = []

# # Convert unique_title_author_pairs to a list
# unique_pairs_list = list(unique_title_author_pairs)

# # Loop through the pairs
# for title, author in unique_pairs_list:
#     retry_count = 0
#     max_retries = 3
#     found = False

#     while retry_count < max_retries and not found:
#         try:
#             # Remove content within parentheses and punctuation from the title
#             title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()
#             title_without_punctuation = title_without_parentheses.translate(str.maketrans('', '', string.punctuation))

#             # Concatenate the title and author's last name
#             search_query = f"{title_without_punctuation} {author.split()[-1]}"

#             # Look up the book page URL for the given title and author
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(search_query)  # Use modified search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(2)

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             # Iterate through the link elements
#             for link_element in link_elements:
#                 # Remove content within parentheses from the link element text
#                 link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

#                 # Check if the modified title is present in the modified link element text
#                 if title_without_parentheses.lower() in link_text_without_parentheses.lower():
#                     # Check if the link text contains any excluded keywords
#                     excluded_keywords = ['summary', 'analysis', 'review', 'synopsis', 'study']
#                     if any(keyword in link_text_without_parentheses.lower() for keyword in excluded_keywords):
#                         continue  # Skip this link element if it contains excluded keywords

#                     # Click on the link element
#                     link_element.click()
#                     found = True
#                     break

#             # Wait for the book page to load
#             time.sleep(2)

#             # Check if the author's last name is present on the book page
#             try:
#                 author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
#             except NoSuchElementException:
#                 author_element = None

#             if author_element:
#                 soup = BeautifulSoup(driver.page_source, 'html.parser')
#                 gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#                 gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                 avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                 count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#                 count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#                 date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#                 # Scrape the genre
#                 genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#                 genres = [genre.text.strip() for genre in genre_elements]
#                 genre1 = ', '.join(genres) if genres else None

#                 # Append scraped data to the final_found list
#                 final_found.append({
#                     'title': gr_title,
#                     'author': gr_author,
#                     'avg_rating': avg_rating,
#                     'count_rating': count_rating,
#                     'count_reviews': count_reviews,
#                     'date_published': date_published,
#                     'top_genre': genre1
#                 })
#                 break

#         except WebDriverException as e:
#             if "502 Bad Gateway" in str(e):
#                 print("502 Bad Gateway encountered. Retrying...")
#             else:
#                 print(f"Error occurred: {str(e)}")
#                 break  # Exit the retry loop if any other exception occurs

#         retry_count += 1

#     if not found:
#         # Handling the case when no matching title element is found after the retry attempts
#         final_unavailable.append({
#             'title': title,
#             'author': author
#         })


Error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="q"]"}
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x011FA813+48355]
	(No symbol) [0x0118C4B1]
	(No symbol) [0x01095358]
	(No symbol) [0x010C09A5]
	(No symbol) [0x010C0B3B]
	(No symbol) [0x010EE232]
	(No symbol) [0x010DA784]
	(No symbol) [0x010EC922]
	(No symbol) [0x010DA536]
	(No symbol) [0x010B82DC]
	(No symbol) [0x010B93DD]
	GetHandleVerifier [0x0145AABD+2539405]
	GetHandleVerifier [0x0149A78F+2800735]
	GetHandleVerifier [0x0149456C+2775612]
	GetHandleVerifier [0x012851E0+616112]
	(No symbol) [0x01195F8C]
	(No symbol) [0x01192328]
	(No symbol) [0x0119240B]
	(No symbol) [0x01184FF7]
	BaseThreadInitThunk [0x76037D59+25]
	RtlInitializeExceptionChain [0x7799B74B+107]
	RtlClearBits [0x7799B6CF+191]

Error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="q"]"}
  (Session info: chrome=1

In [191]:
# len(final_found)

165

In [192]:
# len(final_unavailable)

892

In [193]:
# # Convert unique_title_author_pairs to a list
# # Add 'discussion' to word list
# # try searching for author name on search page instead of book page
# unique_pairs_list = list(unique_title_author_pairs)

# # Loop through the pairs
# for title, author in unique_pairs_list[165:175]:
#     retry_count = 0
#     max_retries = 3
#     found = False

#     while retry_count < max_retries and not found:
#         try:
#             # Remove content within parentheses and punctuation from the title
#             title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()
#             title_without_punctuation = title_without_parentheses.translate(str.maketrans('', '', string.punctuation))

#             # Concatenate the title and author's last name
#             search_query = f"{title_without_punctuation} {author.split()[-1]}"

#             # Look up the book page URL for the given title and author
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(search_query)  # Use modified search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(3)

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             # Iterate through the link elements
#             for link_element in link_elements:
#                 # Remove content within parentheses from the link element text
#                 link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

#                 # Check if the modified title is present in the modified link element text
#                 if title_without_parentheses.lower() in link_text_without_parentheses.lower():
#                     # Check if the link text contains any excluded keywords
#                     excluded_keywords = ['summary', 'analysis', 'review', 'synopsis', 'study']
#                     if any(keyword in link_text_without_parentheses.lower() for keyword in excluded_keywords):
#                         continue  # Skip this link element if it contains excluded keywords

#                     # Click on the link element
#                     link_element.click()

#                     # Wait for the book page to load
#                     time.sleep(3)

#                     # Check if the author's last name is present on the book page
#                     author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
#                     if author_element:
#                         soup = BeautifulSoup(driver.page_source, 'html.parser')
#                         gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#                     gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                     avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                     count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#                     count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#                     date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#                      # Scrape the genre
#                     genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#                     genres = [genre.text.strip() for genre in genre_elements]
#                     genre1 = ', '.join(genres) if genres else None

#                     # Append scraped data to the final_found list
#                     final_found.append({
#                     'title': gr_title,
#                     'author': gr_author,
#                     'avg_rating': avg_rating,
#                     'count_rating': count_rating,
#                     'count_reviews': count_reviews,
#                     'date_published': date_published,
#                     'top_genre': genre1
#                 })
#                 break

#         except WebDriverException as e:
#             if "502 Bad Gateway" in str(e):
#                 print("502 Bad Gateway encountered. Retrying...")
#             else:
#                 print(f"Error occurred: {str(e)}")
#                 break  # Exit the retry loop if any other exception occurs

#         retry_count += 1

#     if not found:
#         # Handling the case when no matching title element is found after the retry attempts
#         final_unavailable.append({
#             'title': title,
#             'author': author
#         })

KeyboardInterrupt: 

In [194]:
# #that didn't work, just kept searching, never clicking any link
# #going to go back to formerly working one, add discussion to word list

# # Convert unique_title_author_pairs to a list
# unique_pairs_list = list(unique_title_author_pairs)

# # Loop through the pairs
# for title, author in unique_pairs_list[165:]:
#     retry_count = 0
#     max_retries = 3
#     found = False

#     while retry_count < max_retries and not found:
#         try:
#             # Remove content within parentheses and punctuation from the title
#             title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()
#             title_without_punctuation = title_without_parentheses.translate(str.maketrans('', '', string.punctuation))

#             # Concatenate the title and author's last name
#             search_query = f"{title_without_punctuation} {author.split()[-1]}"

#             # Look up the book page URL for the given title and author
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(search_query)  # Use modified search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(2)

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             # Iterate through the link elements
#             for link_element in link_elements:
#                 # Remove content within parentheses from the link element text
#                 link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

#                 # Check if the modified title is present in the modified link element text
#                 if title_without_parentheses.lower() in link_text_without_parentheses.lower():
#                     # Check if the link text contains any excluded keywords
#                     excluded_keywords = ['summary', 'analysis', 'review', 'synopsis', 'study', 'discussion']
#                     if any(keyword in link_text_without_parentheses.lower() for keyword in excluded_keywords):
#                         continue  # Skip this link element if it contains excluded keywords

#                     # Click on the link element
#                     link_element.click()
#                     found = True
#                     break

#             # Wait for the book page to load
#             time.sleep(2)

#             # Check if the author's last name is present on the book page
#             try:
#                 author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
#             except NoSuchElementException:
#                 author_element = None

#             if author_element:
#                 soup = BeautifulSoup(driver.page_source, 'html.parser')
#                 gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#                 gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                 avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                 count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#                 count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#                 date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#                 # Scrape the genre
#                 genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#                 genres = [genre.text.strip() for genre in genre_elements]
#                 genre1 = ', '.join(genres) if genres else None

#                 # Append scraped data to the final_found list
#                 final_found.append({
#                     'title': gr_title,
#                     'author': gr_author,
#                     'avg_rating': avg_rating,
#                     'count_rating': count_rating,
#                     'count_reviews': count_reviews,
#                     'date_published': date_published,
#                     'top_genre': genre1
#                 })
#                 break

#         except WebDriverException as e:
#             if "502 Bad Gateway" in str(e):
#                 print("502 Bad Gateway encountered. Retrying...")
#             else:
#                 print(f"Error occurred: {str(e)}")
#                 break  # Exit the retry loop if any other exception occurs

#         retry_count += 1

#     if not found:
#         # Handling the case when no matching title element is found after the retry attempts
#         final_unavailable.append({
#             'title': title,
#             'author': author
#         })



Error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="q"]"}
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x011FA813+48355]
	(No symbol) [0x0118C4B1]
	(No symbol) [0x01095358]
	(No symbol) [0x010C09A5]
	(No symbol) [0x010C0B3B]
	(No symbol) [0x010EE232]
	(No symbol) [0x010DA784]
	(No symbol) [0x010EC922]
	(No symbol) [0x010DA536]
	(No symbol) [0x010B82DC]
	(No symbol) [0x010B93DD]
	GetHandleVerifier [0x0145AABD+2539405]
	GetHandleVerifier [0x0149A78F+2800735]
	GetHandleVerifier [0x0149456C+2775612]
	GetHandleVerifier [0x012851E0+616112]
	(No symbol) [0x01195F8C]
	(No symbol) [0x01192328]
	(No symbol) [0x0119240B]
	(No symbol) [0x01184FF7]
	BaseThreadInitThunk [0x76037D59+25]
	RtlInitializeExceptionChain [0x7799B74B+107]
	RtlClearBits [0x7799B6CF+191]

Error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="q"]"}
  (Session info: chrome=1

In [195]:
# # Add 'journal', 'notes' and 'workbook', 'signed', 'preview' to word list
# #note to self, for future join, remove anything with 0 reviews

# len(final_found)

340

In [196]:
# # Loop through the pairs
# for title, author in unique_pairs_list[340:]:
#     retry_count = 0
#     max_retries = 3
#     found = False

#     while retry_count < max_retries and not found:
#         try:
#             # Remove content within parentheses and punctuation from the title
#             title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()
#             title_without_punctuation = title_without_parentheses.translate(str.maketrans('', '', string.punctuation))

#             # Concatenate the title and author's last name
#             search_query = f"{title_without_punctuation} {author.split()[-1]}"

#             # Look up the book page URL for the given title and author
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(search_query)  # Use modified search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(2)

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             # Iterate through the link elements
#             for link_element in link_elements:
#                 # Remove content within parentheses from the link element text
#                 link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

#                 # Check if the modified title is present in the modified link element text
#                 if title_without_parentheses.lower() in link_text_without_parentheses.lower():
#                     # Check if the link text contains any excluded keywords
#                     excluded_keywords = ['summary', 'analysis', 'review', 'synopsis', 'study', 'discussion', 'journal', 'notes', 'workbook', 'signed', 'preview']
#                     if any(keyword in link_text_without_parentheses.lower() for keyword in excluded_keywords):
#                         continue  # Skip this link element if it contains excluded keywords

#                     # Click on the link element
#                     link_element.click()
#                     found = True
#                     break

#             # Wait for the book page to load
#             time.sleep(2)

#             # Check if the author's last name is present on the book page
#             try:
#                 author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
#             except NoSuchElementException:
#                 author_element = None

#             if author_element:
#                 soup = BeautifulSoup(driver.page_source, 'html.parser')
#                 gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#                 gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                 avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                 count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#                 count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#                 date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#                 # Scrape the genre
#                 genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#                 genres = [genre.text.strip() for genre in genre_elements]
#                 genre1 = ', '.join(genres) if genres else None

#                 # Append scraped data to the final_found list
#                 final_found.append({
#                     'title': gr_title,
#                     'author': gr_author,
#                     'avg_rating': avg_rating,
#                     'count_rating': count_rating,
#                     'count_reviews': count_reviews,
#                     'date_published': date_published,
#                     'top_genre': genre1
#                 })
#                 break

#         except WebDriverException as e:
#             if "502 Bad Gateway" in str(e):
#                 print("502 Bad Gateway encountered. Retrying...")
#             else:
#                 print(f"Error occurred: {str(e)}")
#                 break  # Exit the retry loop if any other exception occurs

#         retry_count += 1

#     if not found:
#         # Handling the case when no matching title element is found after the retry attempts
#         final_unavailable.append({
#             'title': title,
#             'author': author
#         })

Error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="q"]"}
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x011FA813+48355]
	(No symbol) [0x0118C4B1]
	(No symbol) [0x01095358]
	(No symbol) [0x010C09A5]
	(No symbol) [0x010C0B3B]
	(No symbol) [0x010EE232]
	(No symbol) [0x010DA784]
	(No symbol) [0x010EC922]
	(No symbol) [0x010DA536]
	(No symbol) [0x010B82DC]
	(No symbol) [0x010B93DD]
	GetHandleVerifier [0x0145AABD+2539405]
	GetHandleVerifier [0x0149A78F+2800735]
	GetHandleVerifier [0x0149456C+2775612]
	GetHandleVerifier [0x012851E0+616112]
	(No symbol) [0x01195F8C]
	(No symbol) [0x01192328]
	(No symbol) [0x0119240B]
	(No symbol) [0x01184FF7]
	BaseThreadInitThunk [0x76037D59+25]
	RtlInitializeExceptionChain [0x7799B74B+107]
	RtlClearBits [0x7799B6CF+191]

Error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="q"]"}
  (Session info: chrome=1

In [197]:
len(final_found)

368

In [199]:
# # Loop through the pairs
# for title, author in unique_pairs_list[368:]:
#     retry_count = 0
#     max_retries = 3
#     found = False

#     while retry_count < max_retries and not found:
#         try:
#             # Remove content within parentheses and punctuation from the title
#             title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()
#             title_without_punctuation = title_without_parentheses.translate(str.maketrans('', '', string.punctuation))

#             # Concatenate the title and author's last name
#             search_query = f"{title_without_punctuation} {author.split()[-1]}"

#             # Look up the book page URL for the given title and author
#             search_book = driver.find_element(By.NAME, "q")
#             search_book.clear()  # Clear any previous search
#             search_book.send_keys(search_query)  # Use modified search query
#             search_book.submit()

#             # Wait for search results to load
#             time.sleep(2)

#             # Find all the link elements on the search results page
#             link_elements = driver.find_elements(By.TAG_NAME, "a")

#             # Iterate through the link elements
#             for link_element in link_elements:
#                 # Remove content within parentheses from the link element text
#                 link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

#                 # Check if the modified title is present in the modified link element text
#                 if title_without_parentheses.lower() in link_text_without_parentheses.lower():
#                     # Check if the link text contains any excluded keywords
#                     excluded_keywords = ['summary', 'analysis', 'review', 'synopsis', 'study', 'discussion', 'journal', 'notes', 'workbook', 'signed', 'preview']
#                     if any(keyword in link_text_without_parentheses.lower() for keyword in excluded_keywords):
#                         continue  # Skip this link element if it contains excluded keywords

#                     # Click on the link element
#                     link_element.click()
#                     found = True
#                     break

#             # Wait for the book page to load
#             time.sleep(2)

#             # Check if the author's last name is present on the book page
#             try:
#                 author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
#             except NoSuchElementException:
#                 author_element = None

#             if author_element:
#                 soup = BeautifulSoup(driver.page_source, 'html.parser')
#                 gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
#                 gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
#                 avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
#                 count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
#                 count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
#                 date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

#                 # Scrape the genre
#                 genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
#                 genres = [genre.text.strip() for genre in genre_elements]
#                 genre1 = ', '.join(genres) if genres else None

#                 Append scraped data to the final_found list
#                 final_found.append({
#                     'title': gr_title,
#                     'author': gr_author,
#                     'avg_rating': avg_rating,
#                     'count_rating': count_rating,
#                     'count_reviews': count_reviews,
#                     'date_published': date_published,
#                     'top_genre': genre1
#                 })
#                 break

#         except WebDriverException as e:
#             if "502 Bad Gateway" in str(e):
#                 print("502 Bad Gateway encountered. Retrying...")
#             else:
#                 print(f"Error occurred: {str(e)}")
#                 break  # Exit the retry loop if any other exception occurs

#         retry_count += 1

#     if not found:
#         # Handling the case when no matching title element is found after the retry attempts
#         final_unavailable.append({
#             'title': title,
#             'author': author
#         })

Error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="q"]"}
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x011FA813+48355]
	(No symbol) [0x0118C4B1]
	(No symbol) [0x01095358]
	(No symbol) [0x010C09A5]
	(No symbol) [0x010C0B3B]
	(No symbol) [0x010EE232]
	(No symbol) [0x010DA784]
	(No symbol) [0x010EC922]
	(No symbol) [0x010DA536]
	(No symbol) [0x010B82DC]
	(No symbol) [0x010B93DD]
	GetHandleVerifier [0x0145AABD+2539405]
	GetHandleVerifier [0x0149A78F+2800735]
	GetHandleVerifier [0x0149456C+2775612]
	GetHandleVerifier [0x012851E0+616112]
	(No symbol) [0x01195F8C]
	(No symbol) [0x01192328]
	(No symbol) [0x0119240B]
	(No symbol) [0x01184FF7]
	BaseThreadInitThunk [0x76037D59+25]
	RtlInitializeExceptionChain [0x7799B74B+107]
	RtlClearBits [0x7799B6CF+191]

Error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="q"]"}
  (Session info: chrome=1

In [ ]:
#make sure to pull out unique values from final_found list, maybe pull out ones with zero ratings first (those need to go back to final_unavailable pile), then unique in final_found
#add 'paperback', 'author', 'conversation' and 'hardback' to word list

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

final_found = []
final_unavailable = []

# Adjust the timeout value as per your requirement
timeout = 10

for title, author in unique_pairs_list[:10]:
    retry_count = 0
    max_retries = 3
    found = False

    while retry_count < max_retries and not found:
        try:
            # Remove content within parentheses and punctuation from the title
            title_without_parentheses = re.sub(r'\([^)]*\)', '', title).strip()
            title_without_punctuation = title_without_parentheses.translate(str.maketrans('', '', string.punctuation))

            # Concatenate the title and author's last name
            search_query = f"{title_without_punctuation} {author.split()[-1]}"

            # Look up the book page URL for the given title and author
            search_book = driver.find_element(By.NAME, "q")
            search_book.clear()  # Clear any previous search
            search_book.send_keys(search_query)  # Use modified search query
            search_book.submit()

            # Wait for search results to load
            wait = WebDriverWait(driver, timeout)
            search_results_loaded = wait.until(EC.presence_of_element_located((By.TAG_NAME, "a")))

            # Find all the link elements on the search results page
            link_elements = driver.find_elements(By.TAG_NAME, "a")

            # Iterate through the link elements
            for link_element in link_elements:
                # Remove content within parentheses from the link element text
                link_text_without_parentheses = re.sub(r'\([^)]*\)', '', link_element.text).strip()

                # Check if the modified title is present in the modified link element text
                if title_without_parentheses.lower() in link_text_without_parentheses.lower():
                    # Check if the link text contains any excluded keywords
                    excluded_keywords = ['summary', 'analysis', 'review', 'synopsis', 'study', 'discussion', 'journal', 'notes', 'workbook', 'paperback', 'hardback', 'author', 'conversation', 'signed', 'preview']
                    if any(keyword in link_text_without_parentheses.lower() for keyword in excluded_keywords):
                        continue  # Skip this link element if it contains excluded keywords

                    # Click on the link element
                    link_element.click()
                    found = True
                    break

            # Wait for the book page to load
            book_page_loaded = wait.until(EC.presence_of_element_located((By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")))

            # Check if the author's last name is present on the book page
            try:
                author_element = driver.find_element(By.XPATH, f"//span[contains(text(), '{author.split()[-1]}')]")
            except NoSuchElementException:
                author_element = None

            if author_element:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                gr_title = soup.find('h1', {'class': 'Text Text__title1'}).text.strip() if soup.find('h1', {'class': 'Text Text__title1'}) else None
                gr_author = soup.find('span', {'class': 'ContributorLink__name'}).text.strip() if soup.find('span', {'class': 'ContributorLink__name'}) else None
                avg_rating = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip() if soup.find('div', {'class': 'RatingStatistics__rating'}) else None
                count_rating = soup.find('span', {'data-testid': 'ratingsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'ratingsCount'}) else None
                count_reviews = soup.find('span', {'data-testid': 'reviewsCount'}).text.strip().split()[0].replace(',', '') if soup.find('span', {'data-testid': 'reviewsCount'}) else None
                date_published = soup.find('p', {'data-testid': 'publicationInfo'}).text.strip() if soup.find('p', {'data-testid': 'publicationInfo'}) else None

                # Scrape the genre
                genre_elements = soup.select('div.BookPageMetadataSection__genres > ul > span > span')
                genres = [genre.text.strip() for genre in genre_elements]
                genre1 = ', '.join(genres) if genres else None

                # Append scraped data to the final_found list
                final_found.append({
                    'title': gr_title,
                    'author': gr_author,
                    'avg_rating': avg_rating,
                    'count_rating': count_rating,
                    'count_reviews': count_reviews,
                    'date_published': date_published,
                    'top_genre': genre1
                })
                break

        except WebDriverException as e:
            if "502 Bad Gateway" in str(e):
                print("502 Bad Gateway encountered. Retrying...")
            else:
                print(f"Error occurred: {str(e)}")
                break  # Exit the retry loop if any other exception occurs

        except TimeoutException:
            print("Page loading took too long. Retrying...")
            retry_count += 1
            continue

        retry_count += 1

    if not found:
        # Handling the case when no matching title element is found after the retry attempts
        final_unavailable.append({
            'title': title,
            'author': author
        })


In [1]:
len(final_found)

NameError: name 'final_found' is not defined

In [ ]:
#Add a space where there are '-'